# 如何创建自定义内存类
虽然在LangChain中有几种预定义的内存类型，但你很可能想要添加自己的内存类型，以使其最适合你的应用程序。本笔记本介绍如何实现。


在本笔记本中，我们将向`ConversationChain`添加自定义内存类型。为了添加自定义内存类，我们需要导入基本内存类并进行子类化。


In [1]:
from langchain import OpenAI, ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
from typing import List, Dict, Any

在本示例中，我们将编写一个使用spacy提取实体并将关于它们的信息保存在简单哈希表中的自定义内存类。然后，在对话期间，我们将查看输入文本，提取任何实体，并将任何有关它们的信息放入上下文中。

* 请注意，此实现非常简单和脆弱，可能无法在生产环境中使用。它的目的是展示您可以添加自定义内存实现。

为此，我们需要使用spacy。


In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_lg

In [5]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [9]:
class SpacyEntityMemory(BaseMemory, BaseModel):
    """Memory class for storing information about entities."""

    # Define dictionary to store information about entities.
    entities: dict = {}
    # Define key to pass information about entities into prompt.
    memory_key: str = "entities"
        
    def clear(self):
        self.entities = {}

    @property
    def memory_variables(self) -> List[str]:
        """Define the variables we are providing to the prompt."""
        return [self.memory_key]

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Load the memory variables, in this case the entity key."""
        # Get the input text and run through spacy
        doc = nlp(inputs[list(inputs.keys())[0]])
        # Extract known information about entities, if they exist.
        entities = [self.entities[str(ent)] for ent in doc.ents if str(ent) in self.entities]
        # Return combined information about entities to put into context.
        return {self.memory_key: "\n".join(entities)}

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save context from this conversation to buffer."""
        # Get the input text and run through spacy
        text = inputs[list(inputs.keys())[0]]
        doc = nlp(text)
        # For each entity that was mentioned, save this information to the dictionary.
        for ent in doc.ents:
            ent_str = str(ent)
            if ent_str in self.entities:
                self.entities[ent_str] += f"\n{text}"
            else:
                self.entities[ent_str] = text

现在我们定义一个提示，它接收有关实体以及用户输入的信息。

In [10]:
from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. You are provided with information about entities the Human mentions, if relevant.

Relevant entity information:
{entities}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["entities", "input"], template=template
)

现在，我们把它们全部组合起来！


In [11]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, prompt=prompt, verbose=True, memory=SpacyEntityMemory())

在第一个示例中，没有关于Harrison的先前知识，“相关实体信息”部分为空。

In [12]:
conversation.predict(input="Harrison likes machine learning")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. You are provided with information about entities the Human mentions, if relevant.

Relevant entity information:


Conversation:
Human: Harrison likes machine learning
AI:

> Finished ConversationChain chain.


" That's great to hear! Machine learning is a fascinating field of study. It involves using algorithms to analyze data and make predictions. Have you ever studied machine learning, Harrison?"

现在在第二个示例中，我们可以看到它收集了有关Harrison的信息。

In [13]:
conversation.predict(input="What do you think Harrison's favorite subject in college was?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. You are provided with information about entities the Human mentions, if relevant.

Relevant entity information:
Harrison likes machine learning

Conversation:
Human: What do you think Harrison's favorite subject in college was?
AI:

> Finished ConversationChain chain.


' From what I know about Harrison, I believe his favorite subject in college was machine learning. He has expressed a strong interest in the subject and has mentioned it often.'

再次请注意，此实现非常简单和脆弱，可能无法在生产环境中使用。它的目的是展示您可以添加自定义内存实现。
